# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. Daniel Faria

### Grupo 16
<dl>
    <dt>12 horas (33.3%)</dt>
    <dd>ist199656 Diogo Guerreiro</dd>
    <dt>12 horas (33.3%)</dt>
    <dd>ist199642 André Bento</dd>
    <dt>12 horas (33.3%)</dt>
    <dd>ist1103972 Afonso Resendes</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL

);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql
-- populate.sql

INSERT INTO customer(cust_no, name, email, phone, address)
VALUES (1, 'João da Silva', 'joao.silva@email.com', '987654321', '1111-111 Av.A'),
       (2, 'Maria Santos', 'maria.santos@email.com', '912345678', '2222-222 Av.B'),
       (3, 'Pedro Oliveira', 'pedro.oliveira@email.com', '923456789', '3333-333 Av.C'),
       (4, 'Ana Pereira', 'ana.pereira@email.com', '934567890', '4444-444 Av.D'),
       (5, 'Carlos Rodrigues', 'carlos.rodrigues@email.com', '945678901', '5555-555 Av.E'),
       (6, 'Sofia Costa', 'sofia.costa@email.com', '956789012', '6666-666 Av.F'),
       (7, 'Luís Gonçalves', 'luis.goncalves@email.com', '967890123', '7777-777 Av.G'),
       (8, 'Beatriz Almeida', 'beatriz.almeida@email.com', '978901234', '8888-888 Av.H');





INSERT INTO employee(ssn, TIN, bdate, name)
VALUES ('12345678901', '12345678901', '1980-01-01', 'João Santos'),
       ('23456789012', '23456789012', '1985-02-02', 'Maria Silva'),
       ('34567890123', '34567890123', '1990-03-03', 'Pedro Oliveira'),
       ('45678901234', '45678901234', '1995-04-04', 'Ana Pereira'),
       ('56789012345', '56789012345', '2000-05-05', 'Carlos Rodrigues'),
       ('67890123456', '67890123456', '1988-06-06', 'Sofia Costa'),
       ('78901234567', '78901234567', '1977-07-07', 'Luís Gonçalves'),
       ('89012345678', '89012345678', '1982-08-08', 'Beatriz Almeida');



INSERT INTO department(name)
VALUES ('Finance'),
       ('Sales'),
       ('Marketing'),
       ('HR'),
       ('IT'),
       ('Operations');


CALL insert_workplace('Rua X, 123', -23.5489, -46.6388,TRUE,FALSE);
       CALL insert_workplace('Av. Y, 456', -22.9068, -43.1729,TRUE,FALSE);
       CALL insert_workplace('Rua Z, 789', -15.7801, -47.9292,TRUE,FALSE);
       CALL insert_workplace('Av. W, 987', -30.0331, -51.2266,TRUE,FALSE);
       CALL insert_workplace('Rua V, 654', -25.4284, -49.2733,FALSE,TRUE);
       CALL insert_workplace('Av. U, 321', -23.5505, -46.6333,FALSE,TRUE);
       CALL insert_workplace('Rua T, 789', -19.9167, -43.9345,FALSE,TRUE);
       CALL insert_workplace('Av. S, 654', -3.7319, -38.5267,FALSE,TRUE);

   
INSERT INTO works(ssn, name, address)
VALUES ('12345678901', 'Finance', 'Rua X, 123'),
       ('23456789012', 'Sales', 'Av. Y, 456'),
       ('34567890123', 'Marketing', 'Rua Z, 789'),
       ('45678901234', 'HR', 'Av. W, 987'),
       ('56789012345', 'IT', 'Rua V, 654'),
       ('67890123456', 'Operations', 'Av. U, 321'),
       ('78901234567', 'Finance', 'Rua T, 789'),
       ('89012345678', 'Sales', 'Av. S, 654');
       
INSERT INTO product(SKU, name, description, price, ean)
VALUES ('12345', 'Camiseta', 'Camisola branca', 29.90, 7891234567890),
       ('23456', 'Calça Jeans', 'Calças jeans', 99.90, 7892345678901),
       ('34567', 'Sapato', 'Sapatos pretos', 149.90, 7893456789012),
       ('45678', 'Vestido', 'Vestido', 79.90, 7894567890123),
       ('56789', 'Camisa Polo', 'Camisa polo', 59.90, 7895678901234),
       ('67890', 'Bermuda', 'Calcoes jeans', 39.90, 7896789012345),
       ('78901', 'Blusa', 'Casaco', 49.90, 7897890123456),
       ('89012', 'Saia', 'Saia longa', 69.90, 7898901234567);


CALL add_order_with_contains (1, 1, '2022-07-01','12345',2);
CALL add_order_with_contains (2, 3, '2022-07-02', '23456',1);
CALL add_order_with_contains (3, 2, '2022-05-03','34567',3);
CALL add_order_with_contains (4, 5, '2022-05-04', '45678',1);
CALL add_order_with_contains  (5, 4, '2022-05-05','56789',2);
CALL add_order_with_contains (6, 6, '2022-05-06','67890',1);
CALL add_order_with_contains (7, 1, '2022-06-07','78901',4);
CALL add_order_with_contains (8, 7, '2022-06-08','89012',3);




INSERT INTO supplier(TIN, name, address, SKU, date)
VALUES ('12345678901', 'Fornecedor A', 'Rua A, 123', '12345', '2022-07-01'),
       ('23456789012', 'Fornecedor B', 'Av. B, 456', '23456', '2022-07-02'),
       ('34567890123', 'Fornecedor C', 'Rua C, 789', '34567', '2022-05-03'),
       ('45678901234', 'Fornecedor D', 'Av. D, 987', '45678', '2022-05-04'),
       ('56789012345', 'Fornecedor E', 'Rua E, 654', '56789', '2022-05-05'),
       ('67890123456', 'Fornecedor F', 'Av. F, 321', '67890', '2022-05-06'),
       ('78901234567', 'Fornecedor G', 'Rua G, 789', '78901', '2022-06-07'),
       ('89012345678', 'Fornecedor H', 'Av. H, 654', '89012', '2022-06-08');

INSERT INTO delivery(address, TIN)
VALUES ('Rua V, 654', '56789012345'),
       ('Av. U, 321', '67890123456'),
       ('Rua T, 789', '78901234567'),
       ('Av. S, 654', '89012345678');
       
       
INSERT INTO process(ssn, order_no)
VALUES ('12345678901', 1),
       ('12345678901', 2),
       ('12345678901', 3),
       ('12345678901', 4),
       ('12345678901', 5),
       ('12345678901', 6),
       ('12345678901', 7),
       ('12345678901', 8);
       
INSERT INTO pay(order_no, cust_no)
VALUES (1, 1),
       (2, 3),
       (3, 2),
       (4, 5),
       (6, 6),
       (7, 1),
       (8, 7);


## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql
-- (RI-1)

CREATE OR REPLACE FUNCTION check_employee_age()
RETURNS TRIGGER AS $$
DECLARE
    employee_age INTEGER;
BEGIN

    employee_age := EXTRACT(YEAR FROM age(NEW.bdate));


    IF employee_age < 18 THEN
        RAISE EXCEPTION 'A idade do empregado deve ser maior ou igual a 18 anos.';
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;


CREATE TRIGGER check_employee_age_trigger
BEFORE INSERT OR UPDATE ON employee
FOR EACH ROW
EXECUTE FUNCTION check_employee_age();



(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql
-- (RI-2)

CREATE OR REPLACE PROCEDURE insert_workplace(
  p_address VARCHAR(255),
  p_lat NUMERIC(8, 6),
  p_long NUMERIC(9, 6),
  p_office BOOLEAN,
  p_warehouse BOOLEAN
)
AS $$
BEGIN
  IF (p_office = p_warehouse) THEN
    RAISE EXCEPTION 'Somente um tipo de Workplace (Office ou Warehouse) pode ser especificado.';
  END IF;

  INSERT INTO workplace (address, lat, long)
  VALUES (p_address, p_lat, p_long);

  IF (p_office = TRUE) THEN
    INSERT INTO office (address)
    VALUES (p_address);
  END IF;

  IF (p_warehouse = TRUE) THEN
    INSERT INTO warehouse (address)
    VALUES (p_address);
  END IF;
END;
$$ LANGUAGE plpgsql;


CREATE OR REPLACE PROCEDURE remove_workplace(
  p_address VARCHAR(255)
)
AS $$
BEGIN
  IF NOT EXISTS (SELECT 1 FROM workplace WHERE address = p_address) THEN
    RAISE EXCEPTION 'Endereço de Workplace inválido.';
  END IF;

  DELETE FROM office WHERE address = p_address;

  DELETE FROM warehouse WHERE address = p_address;

  DELETE FROM workplace WHERE address = p_address;
END;
$$ LANGUAGE plpgsql;





(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql
-- (RI-3)

CREATE OR REPLACE PROCEDURE add_order_with_contains(
  p_order_no INTEGER,
  p_cust_no INTEGER,
  p_date DATE,
  p_sku VARCHAR(25),
  p_qty INTEGER
)
AS $$
BEGIN
  INSERT INTO orders (order_no, cust_no, date)
  VALUES (p_order_no, p_cust_no, p_date);

  INSERT INTO contains (order_no, SKU, qty)
  VALUES (p_order_no, p_sku, p_qty);
  
  COMMIT;
END;
$$ LANGUAGE plpgsql;



## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql

SELECT c.cust_no, c.name, SUM(p.price * co.qty) AS total_order_value
FROM customer c
JOIN pay pa ON c.cust_no = pa.cust_no
JOIN orders o ON pa.order_no = o.order_no
JOIN contains co ON o.order_no = co.order_no
JOIN product p ON co.SKU = p.SKU
GROUP BY c.cust_no, c.name
HAVING SUM(p.price * co.qty) = (
  SELECT MAX(total_order_value)
  FROM (
    SELECT SUM(p.price * co.qty) AS total_order_value
    FROM customer c
    JOIN pay pa ON c.cust_no = pa.cust_no
    JOIN orders o ON pa.order_no = o.order_no
    JOIN contains co ON o.order_no = co.order_no
    JOIN product p ON co.SKU = p.SKU
    GROUP BY c.cust_no
  ) AS subquery
);



2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql


SELECT e.name
FROM employee e
WHERE NOT EXISTS (
  SELECT DISTINCT o.date
  FROM orders o
  WHERE o.date BETWEEN '2022-01-01' AND '2022-12-31'
    AND NOT EXISTS (
      SELECT 1
      FROM process p
      WHERE p.ssn = e.ssn AND p.order_no = o.order_no
    )
)





3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql


SELECT EXTRACT(MONTH FROM o.date) AS month, COUNT(*) AS unpaid_orders
FROM orders o
LEFT JOIN pay p ON o.order_no = p.order_no
WHERE p.order_no IS NULL AND EXTRACT(YEAR FROM o.date) = 2022
GROUP BY month
ORDER BY month;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql

CREATE VIEW product_sales AS
SELECT c.SKU, c.order_no, c.qty, (c.qty * p.price) AS total_price,
       EXTRACT(YEAR FROM o.date) AS year, EXTRACT(MONTH FROM o.date) AS month,
       EXTRACT(DAY FROM o.date) AS day_of_month, EXTRACT(DOW FROM o.date) AS day_of_week,
       SUBSTRING(cust.address, 10) AS city
FROM contains c
JOIN orders o ON c.order_no = o.order_no
JOIN customer cust ON o.cust_no = cust.cust_no
JOIN product p ON c.SKU = p.SKU
JOIN pay py ON o.order_no = py.order_no;

## 4. Desenvolvimento de Aplicação

A arquitetura do app consiste em um ficheiro Python chamado "app" que define uma aplicação Flask para gerenciar um sistema de produtos, clientes, fornecedores e pedidos. Este ficheiro possui rotas para executar operações de criação, leitura, atualização e exclusão (CRUD) no banco de dados PostgreSQL, além de renderizar templates HTML.
A app também possui uma pasta chamada "templates" que contém o ficheiro "base.html", onde está presente a barra lateral. Dentro dessa pasta, existem quatro subpastas ("customer", "order", "product" e "supplier"), cada uma contendo ficheiros HTML relacionados ao seu respectivo nome. Estes ficheiros podem ser "index" (que exibe a lista de todos os elementos), "edit" (que permite a edição) e "create" (que permite a criação).
Além disso, há uma pasta chamada "static" que contém ficheiros CSS para estilizar todo o site.


## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql


SELECT sku, city, month, day_of_month, day_of_week, SUM(qty) AS total_qty, SUM(total_price) AS total_price
FROM product_sales
WHERE year = 2022
GROUP BY GROUPING SETS ((city,sku ), (month, sku), (day_of_month, sku), (day_of_week, sku))
ORDER BY sku, city, month, day_of_month, day_of_week;

2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql

CREATE TABLE data (
  date_id SERIAL PRIMARY KEY,
  date_value DATE,
  year INT,
  month INT,
  day INT,
  day_of_week INT
);


INSERT INTO data (date_value, year, month, day, day_of_week)
SELECT
  generate_series('2022-01-01'::DATE, '2022-12-31'::DATE, '1 day'),
  EXTRACT(YEAR FROM generate_series),
  EXTRACT(MONTH FROM generate_series),
  EXTRACT(DAY FROM generate_series),
  EXTRACT(DOW FROM generate_series)
FROM generate_series('2022-01-01'::DATE, '2022-12-31'::DATE, '1 day');


In [ ]:
%%sql

SELECT d.month, d.day_of_week, AVG(ps.total_price) AS average_daily_sales
FROM data d
LEFT JOIN product_sales ps ON d.date_value = ps.date
WHERE d.year = 2022
GROUP BY  GROUPING SETS (d.month), (d.day_of_week)
ORDER BY d.month, d.day_of_week;


## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação

Um índice na tabela "product" no atributo "price". Isso ajudará a otimizar a operação de filtro "price > 50" no JOIN com a tabela "contains", permitindo uma busca mais eficiente nos valores de preço.

CREATE INDEX idx_product_price ON product (price);

Um índice na tabela "orders" no atributo "date". Isso ajudará a otimizar a operação de filtro "EXTRACT(YEAR FROM date) = 2023", permitindo uma busca mais rápida nas datas correspondentes ao ano de 2023.

CREATE INDEX idx_orders_date ON orders (date);


### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;


### Tipo de Índice, Atributos & Justificação

Para melhorar o desempenho da consulta apresentada, uma estratégia de indexação adequada seria criar um índice no atributo name da tabela product. Como a consulta está filtrando os produtos pelo prefixo "A%" no atributo name, um índice de pesquisa por prefixo seria eficaz para otimizar essa operação.
Esse índice permitirá que o banco de dados pesquise rapidamente os produtos que possuam o prefixo "A%" no atributo name, reduzindo o tempo de execução da consulta.

CREATE INDEX idx_product_name ON product (name);
